
<a href="https://colab.research.google.com/github/takzen/pytorch-black-belt/blob/main/38_TensorBoard_Logging.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🥋 Lekcja 38: TensorBoard & Logging (Oczy Inżyniera)

Kiedy trenujesz model przez 3 dni, nie chcesz patrzeć na tekstowe logi w konsoli.
Chcesz widzieć wykresy:
*   Czy Loss spada?
*   Czy Accuracy rośnie?
*   Czy histogramy wag wyglądają zdrowo (Gausowsko)?

**TensorBoard** to narzędzie do wizualizacji, które działa w przeglądarce.
PyTorch Lightning integruje się z nim automatycznie. Wystarczy użyć metody `self.log()`.

W tej lekcji nauczymy się logować nie tylko liczby, ale też **Histogramy Wag** i **Graf Modelu**.

In [2]:
# Instalacja TensorBoard (jeśli nie masz)
# !uv add tensorboard

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger

# Konfiguracja
BATCH_SIZE = 64
torch.set_float32_matmul_precision('medium') # Fix dla Twojego RTX 4060!

## Rozbudowany Model z Logowaniem

Zmodyfikujemy nasz model z poprzedniej lekcji.
1.  Dodamy `log_graph=True` w loggerze.
2.  Użyjemy `self.logger.experiment.add_histogram`, żeby śledzić, czy wagi nie wybuchają.

In [4]:
class VisModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        # Zapisujemy przykładowe wejście, żeby TensorBoard mógł narysować graf
        self.example_input_array = torch.randn(1, 10)

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        
        # 1. Logowanie Skalara (Wykres liniowy)
        # on_step=True: Rysuj kropkę co każdy batch (poszarpany wykres)
        # on_epoch=True: Rysuj średnią co epokę (gładki wykres)
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        
        return loss

    def on_train_epoch_end(self):
        # 2. Logowanie Histogramu Wag (Co epokę)
        # Dostęp do "surowego" obiektu TensorBoard
        tensorboard = self.logger.experiment
        
        for name, params in self.named_parameters():
            tensorboard.add_histogram(name, params, self.current_epoch)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

print("Model gotowy do obserwacji.")

Model gotowy do obserwacji.


In [8]:
# Dane (Szum)
dataset = TensorDataset(torch.randn(1000, 10), torch.randn(1000, 1))
loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=0)

# Logger
logger = TensorBoardLogger(save_dir=".", name="moj_eksperyment")

# Trener
trainer = L.Trainer(
    max_epochs=5,
    logger=logger,
    enable_checkpointing=False,
    accelerator="auto",
    devices=1,
    # --- POPRAWKA ---
    # Loguj co 1 krok (bo mamy malutki dataset i kroki lecą szybko)
    log_every_n_steps=1
)

print("🚀 Start treningu z wizualizacją...")
trainer.fit(model=VisModel(), train_dataloaders=loader)
print(f"✅ Koniec. Logi zapisane w folderze: {logger.log_dir}")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode  | FLOPs | In sizes | Out sizes
---------------------------------------------------------------------------
0 | net  | Sequential | 4.9 K  | train | 9.6 K | [1, 10]  | [1, 1]   
---------------------------------------------------------------------------
4.9 K     Trainable params
0         Non-trainable params
4.9 K     Total params
0.020     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode
9.6 K     Total Flops


🚀 Start treningu z wizualizacją...
Epoch 4: 100%|██████████| 16/16 [00:00<00:00, 244.43it/s, v_num=1, train_loss=0.996]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 16/16 [00:00<00:00, 208.61it/s, v_num=1, train_loss=0.996]
✅ Koniec. Logi zapisane w folderze: .\moj_eksperyment\version_1


## Jak zobaczyć wyniki?

TensorBoard to osobna aplikacja webowa. Aby ją uruchomić, masz dwie opcje:

**Opcja A: Wewnątrz Notebooka (Magic Command)**
```python
%load_ext tensorboard
%tensorboard --logdir .
```
**Opcja B: W terminalu (VS Code)**
Otwórz terminal w folderze projektu i wpisz:
```python
tensorboard --logdir .
```
A potem wejdź w przeglądarce na http://localhost:6006.
Tam zobaczysz zakładki:
Scalars: Wykres train_loss.
Graphs: Schemat Twojej sieci neuronowej.
Histograms: Rozkład wag (czy są "zdrowe", czy np. wszystkie zbiegły do zera).

In [9]:
# Próba uruchomienia w notatniku (może wymagać restartu kernela jeśli masz extensions issue)
# Jeśli to nie zadziała, użyj terminala!

try:
    %load_ext tensorboard
    %tensorboard --logdir .
except Exception as e:
    print(f"Nie udało się uruchomić w notatniku: {e}")
    print("Uruchom w terminalu: tensorboard --logdir .")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 27028), started 0:01:26 ago. (Use '!kill 27028' to kill it.)

## 🥋 Black Belt Summary

1.  **Logger:** W Lightning logger jest oddzielony od modelu. Możesz zamienić `TensorBoardLogger` na `WandbLogger` (Weights & Biases) lub `CSVLogger` jedną linijką.
2.  **Artifacts:** Oprócz `loss`, warto logować histogramy wag.
    *   Jeśli histogram "rozlewa się" bardzo szeroko -> Eksplodujące gradienty.
    *   Jeśli histogram kurczy się do cienkiej szpilki na zerze -> Znikające gradienty (albo za duże L2).
3.  **Graf:** Dzięki `example_input_array` TensorBoard narysuje Ci architekturę sieci. To super przydatne przy debugowaniu skomplikowanych modeli.